# Clustering Crypto

In [18]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [19]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv("Resources/crypto_data.csv", index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [20]:
# Keep all the cryptocurrencies that are being traded.
# https://www.statology.org/pandas-drop-rows-with-value/
crypto_df = crypto_df[crypto_df.IsTrading != False]        
crypto_df        

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [21]:
# Remove the "IsTrading" column. 
new_crypto_df = crypto_df.drop(["IsTrading"], axis=1)
new_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [22]:
# Remove rows that have at least 1 null value.
for column in new_crypto_df.columns:
    print(f"Column {column} has {new_crypto_df[column].isnull().sum()}null values")
new_crypto_df = new_crypto_df.dropna()    

Column CoinName has 0null values
Column Algorithm has 0null values
Column ProofType has 0null values
Column TotalCoinsMined has 459null values
Column TotalCoinSupply has 0null values


In [23]:
# Keep the rows where coins are mined.
new_crypto_df = new_crypto_df[new_crypto_df.TotalCoinsMined != 0]        
new_crypto_df 

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [24]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_df = new_crypto_df[["CoinName"]]
crypto_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [25]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
new_crypto_df = new_crypto_df.drop(["CoinName"], axis=1)
new_crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [9]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(new_crypto_df, columns=['Algorithm', 'ProofType'])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11674788 -0.15286468 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.09358885 -0.14499604 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [ 0.52587231  4.4937636  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 ...
 [-0.09523411 -0.13215444 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.11658774 -0.15255408 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.11674507 -0.15284989 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [31]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model 
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [38]:
# Create a DataFrame with the three principal components. https://stackoverflow.com/questions/43798500/how-to-preserve-original-indexes-in-the-new-dataframe
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.325473,1.012374,-0.492133
404,-0.308795,1.012595,-0.492683
1337,2.300918,1.724821,-0.640563
BTC,-0.150387,-1.275081,0.168502
ETH,-0.154833,-1.955675,0.366329


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [44]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\c_tal\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [48]:
# Initialize and fit the K-Means model.
model = KMeans(n_clusters=4, random_state=42).fit(pcs_df)

# Predict clusters
y_pred = model.predict(pcs_df)

In [53]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns. https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html?highlight=drop
clustered_df = pd.concat([new_crypto_df, pcs_df], axis=1).reindex(new_crypto_df.index)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. https://stackoverflow.com/questions/33497896/how-can-i-add-a-column-from-one-dataframe-to-another-dataframe
clustered_df["CoinName"] = crypto_df["CoinName"].values

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = y_pred

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.325473,1.012374,-0.492133,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.308795,1.012595,-0.492683,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.300918,1.724821,-0.640563,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.150387,-1.275081,0.168502,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.154833,-1.955675,0.366329,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.157723,-1.125593,0.059655,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.402009,1.251386,-0.505584,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.151133,-2.206904,0.392132,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.153274,-1.955757,0.366309,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.144489,-2.230794,0.422623,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
